We investigate here the returns of a strategy consisting of buying coins on the day they cross for the first time the rank 200 and hold them for a while. We find out that this strategy was more profitable that holding btc alone, at least for the set of hyperparameters used in this notebook. Note that the strategy is evaluated on profitably alone here, not on risk.

In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pdb
from IPython.display import display, HTML


In [5]:
df = pd.read_csv("all_coins.csv")
df["marketcap"] = df["marketcap"].interpolate(method="nearest").fillna(0)

In [6]:
df.head()

,date,open*,high,low,close**,volume,marketcap,slug,symbol
0,2018-06-26,6253.55,6290.16,6093.67,6093.67,3.279760e+09,1.070330e+11,bitcoin,BTC
1,2018-06-25,6171.97,6327.37,6119.68,6249.18,5.500810e+09,1.056250e+11,bitcoin,BTC
2,2018-06-24,6164.28,6223.78,5826.41,6173.23,4.566910e+09,1.054800e+11,bitcoin,BTC
3,2018-06-23,6090.10,6224.82,6071.81,6162.48,3.431360e+09,1.041980e+11,bitcoin,BTC
4,2018-06-22,6737.88,6747.08,6006.60,6083.69,5.079810e+09,1.152690e+11,bitcoin,BTC


In [8]:
# 1) add rank
dg = pd.DataFrame()
piv = pd.DataFrame()
for n,g in tqdm(df.groupby("date")):
    g["mc_rank"] = g["marketcap"].rank(ascending=False, method="first")
    dg=dg.append(g)

  0%|          | 0/1886 [00:00<?, ?it/s]/Users/greghor/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 1886/1886 [02:20<00:00,  5.14it/s]


In [9]:
#2) add new comer flag based on thresholds (typed as list so as we can add several flags at once)

def add_new_comer_flag(dg, thresholds):
    """
    parameters
    ----------
    df: dataframe with ranks
    thresholds: list, rank threshold for new comer flags
    
    return:
    -------
    dataframe with new comer flags
    """
    
    dh = pd.DataFrame()
    for n, g in tqdm(dg.groupby("symbol")):
        g = g.sort_values("date")
        g["highest_rank"] = np.minimum.accumulate(g["mc_rank"])
        for t in thresholds:
            col_name = "new_comer_" + str(t)
            g[col_name] = 0
            g.loc[(g["highest_rank"].shift(1) > t) & (g["mc_rank"] <= t), col_name] = 1
        dh = dh.append(g)
    return dh

In [ ]:
dh = add_new_comer_flag(dg, [100, 150, 200])

 42%|████▏     | 652/1551 [00:45<01:35,  9.43it/s]

In [171]:

#3) simulate returns
def assets_return(df, threshold, start_buy_date, stop_buy_date, exit_date):
    """
    
    simulate the return of a portfolio that buy asset whenever it crosses the threshold rank for the first time
    and sell the asset at exit date. Every asset get the same allocation
    
    parameters
    ----------
    df: dataframe with with new comer flags
    threshold: threshold rank
    start_buy_date: string, first day of buying period
    stop_buy_date: string, last day of buying period
    exit_date: string, sell date
    
    return
    ------
    dataframe with return for every asset as well as buy/sell price/date
    
    """
    
    if stop_buy_date > exit_date:
        raise ValueError("stop_buy_date must be smaller than exit_date")
    
    cols = ["date", "symbol", "close**"]
    buy_period = (df.date >= start_buy_date) & (df.date < stop_buy_date) 
    exit_cond = df.date == exit_date
    assets_buy_price = df.loc[ buy_period & (df["new_comer_" + str(threshold)] == 1), cols]
    ret = pd.merge(assets_buy_price, df.loc[exit_cond, cols], left_on="symbol", right_on="symbol",
                                                  suffixes=["_buy", "_sell"])
    ret["return"] = (ret["close**_sell"]) / ret["close**_buy"] - 1
    
    return ret

# result are benchmarked against btc
def benchmark_return(df, entry_date, exit_date):
    """
    
    compute the return of a btc portfolio
    
    parameters
    ----------
    df: dataframe with date, price 
    entry_date: buy date
    exit_date: sell date
    
    return
    ------
    btc portfolio return 
    """
    
    in_cond = (df.symbol=="BTC") & (df.date == entry_date)
    out_cond = (df.symbol=="BTC") & (df.date == exit_date)
    return (df.loc[out_cond, "close**"].values[0]) / df.loc[in_cond, "close**"].values[0] - 1

Let's buy assets that cross the rank 200 in 2017 and exit on 2018-01-01

In [177]:
entry_date = "2017-01-01"
exit_date = "2018-01-01"
ret = assets_return(dh, 100, entry_date, exit_date, exit_date)


In [179]:
ret.sort_values("return").reset_index().describe()

,index,close**_buy,close**_sell,return
count,75.000000,75.000000,75.000000,75.000000
mean,37.000000,1.611377,9.068257,36.339189
std,21.794495,4.957575,45.817331,174.677334
min,0.000000,0.000011,0.000004,-0.999911
25%,18.500000,0.082710,0.092506,-0.189203
50%,37.000000,0.380666,0.576237,0.372323
75%,55.500000,1.236790,2.788920,2.922483
max,74.000000,41.256400,385.497000,1053.210917


96 assets bough in this period, mean return around 3600% (impressive right), however 75% of the assets have less than 290% returns. Mean return is taking off because of few skyrocketters

In the mean period the btc return is about 1250% (see below)

In [181]:
benchmark_return(dh, entry_date, exit_date)

12.680114191270379

Things don't look that great if we do the same experience for 2018, but we still beat the bitcoin. 
Again, thanks to few big winners.


In [185]:
entry_date = "2017-06-01"
exit_date = "2018-06-26"
ret = assets_return(dh, 100, entry_date, exit_date, exit_date)
print "portfolio"
display(ret.describe())
print "\n"
print "benchmark"
benchmark_return(dh, entry_date, exit_date)


portfolio


,close**_buy,close**_sell,return
count,105.000000,1.050000e+02,105.000000
mean,6.001100,8.705027e-01,0.174840
std,40.840758,2.936262e+00,4.787445
min,0.000048,5.377000e-07,-0.999979
25%,0.132718,1.988690e-02,-0.900488
50%,0.336202,7.511160e-02,-0.773414
75%,1.586720,3.281790e-01,-0.511277
max,415.927000,1.921140e+01,34.967924




benchmark


1.5307199694336928

A risk metric shall be added in the next steps.